In [1]:
import sys
sys.path.append('/accounts/grad/zhangyunzhe2023/tlide')

from typing import List, Optional
from llama import Dialog, Llama

import os
import torch
import torch.distributed as dist

# choose any free port
os.environ.setdefault("MASTER_ADDR", "127.0.0.1")
os.environ.setdefault("MASTER_PORT", "29500")
os.environ.setdefault("RANK", "0")
os.environ.setdefault("WORLD_SIZE", "1")
os.environ.setdefault("LOCAL_RANK", "0")

# if you have a CUDA GPU, use 'nccl'; otherwise use 'gloo'
backend = "nccl" if torch.cuda.is_available() else "gloo"
if backend == "nccl":
    torch.cuda.set_device(0)

dist.init_process_group(backend=backend)

In [2]:
generator = Llama.build(
    ckpt_dir='/accounts/grad/zhangyunzhe2023/.llama/checkpoints/Llama3.2-1B',
    tokenizer_path='/accounts/grad/zhangyunzhe2023/.llama/checkpoints/Llama3.2-1B/tokenizer.model',
    max_seq_len=8192,
    max_batch_size=1,
)

tokenizer = generator.tokenizer
model = generator.model.eval()
params = generator.model.params

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


/scratch/users/zhangyunzhe2023/conda/envs/torch28/lib/python3.13/site-packages/torch/__init__.py:1264: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


Loaded in 7.53 seconds


In [3]:
from datasets import load_dataset
dataset = load_dataset('THUDM/LongBench-v2', split='train')

In [4]:
text = dataset[0]['context']
start = text.find('\n')
text = text[start+1:]

prompts: List[str] = [text]

In [5]:
prompt_tokens = [tokenizer.encode(prompts[0], bos=True, eos=False)]
bsz = 1
pad_id = tokenizer.pad_id
tokens = torch.tensor(prompt_tokens, dtype=torch.long, device="cuda")

model.clear_cache()
result = []
for pos in range(10):
    result.append(model.forward(tokens[:, pos:pos+1], pos, return_last_hidden=True))

In [6]:
result

[tensor([[[ 1.5497, -1.5041,  2.7454,  ..., -0.6332,  0.6005, -1.0105]]]),
 tensor([[[ 1.4498,  1.9637,  1.7737,  ..., -5.3823, -4.5258,  0.7892]]]),
 tensor([[[ 1.5561,  2.8498,  1.6540,  ..., -5.0222, -6.0117,  0.2023]]]),
 tensor([[[ 1.6423,  1.3369,  1.3579,  ..., -3.9708, -5.2786,  0.1533]]]),
 tensor([[[ 0.4390,  3.5919,  2.9129,  ..., -2.9547, -5.5047,  2.7879]]]),
 tensor([[[ 1.9188,  2.7424,  2.0267,  ..., -4.7778, -6.0142,  0.5419]]]),
 tensor([[[-1.7211,  3.6688, -0.9305,  ..., -2.4419, -1.9752,  1.2490]]]),
 tensor([[[ 0.7205,  1.4898,  1.1228,  ..., -5.8459, -4.8139, -1.0883]]]),
 tensor([[[ 0.5390,  5.1830,  1.7469,  ..., -6.2586, -4.5494, -3.0995]]]),
 tensor([[[ 1.2807,  1.1994,  4.4573,  ..., -5.2102, -2.5543, -1.8037]]])]